In [1]:
import warnings
warnings.filterwarnings("ignore")

from data_loading import *

from xai_agg.agg_exp import *
from xai_agg.utils import *

import pandas as pd
import numpy as np

from scipy.stats import pearsonr, spearmanr, kendalltau

2025-01-16 14:06:01.390083: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-16 14:06:01.413689: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dataset_name, preprocessed_data, categorical_features, X, y, X_train, X_test, y_train, y_test, clf = load_pakdd2010_data()

Accuracy: 0.7466666666666667
ROC AUC: 0.5158014399393709


In [3]:
evaluator = ExplanationModelEvaluator(clf, X_train, categorical_features)
evaluator.init()

Epoch 1/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2481 - val_loss: 1.0735
Epoch 2/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - loss: 1.3362 - val_loss: 1.0234
Epoch 3/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - loss: 1.1877 - val_loss: 0.9750
Epoch 4/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - loss: 1.0771 - val_loss: 0.9349
Epoch 5/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step - loss: 1.0896 - val_loss: 0.9042
Epoch 6/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step - loss: 0.9829 - val_loss: 0.8814
Epoch 7/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step - loss: 0.9799 - val_loss: 0.8644
Epoch 8/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step - loss: 1.0897 - val_loss: 0.8507
Epoch 9/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step - loss: 0.9986 - val_loss: 0.8390
Epoch 10/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - loss: 1.0154 - val_loss: 0.8278
Epoch 11/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step - loss: 0.9178 - val_loss: 0.8175
Epoch 12/500
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 

### Misc

In [4]:
evaluator.faithfullness_correlation(ShapTabularTreeWrapper, X_test.iloc[0])

0.777488762318155

# Generating metric vectors and calculating correlation between them

In [5]:
# Getting a few reanknings
exps = [LimeWrapper(clf, X_train, categorical_features), ShapTabularTreeWrapper(clf, X_train, categorical_features), AnchorWrapper(clf, X_train, categorical_features)]

indexes = np.random.choice(X_test.index, 100, replace=False)


In [6]:
import concurrent.futures

comp_arr = [] # original, new1, new2, ...

def process_index(idx):
    local_comp_arr = []
    for exp in exps:
        row = []
        explanation = exp.explain_instance(X_test.loc[idx])
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=False, , iterations=100))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=False, iterations=100))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="sum", iterations=100))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="percentile", iterations=100))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="avg", iterations=100))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="inverse", iterations=100))
        
        # row.append(evaluator.complexity(exp, X_test.loc[idx], explanation=explanation))
        # row.append(evaluator.nrc(exp, X_test.loc[idx], explanation=explanation))
        
        local_comp_arr.append(row)
    return local_comp_arr
    
with concurrent.futures.ProcessPoolExecutor(5) as executor:
    results = list(executor.map(process_index, indexes))

for result in results:
    comp_arr.extend(result)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now ret

In [7]:
comp_arr

[[0.042115543204383134,
  0.13445546806318182,
  0.11436112086461492,
  0.004476653799282627,
  0.15830155924273018,
  0.08761371994818586],
 [0.44438414286074734,
  0.4905288579636767,
  0.1846125996468011,
  0.023305663290801505,
  0.14152649503780784,
  0.3781719792645691],
 [0.2421357931634488,
  0.1809830944143485,
  0.15659202459607105,
  0.23378478210239495,
  0.11329380276126018,
  0.3280958031604405],
 [0.12546042965181842,
  0.1007693722381217,
  0.40954456566364494,
  0.029202581414015372,
  0.14651027960845947,
  0.07926901605266641],
 [0.2851447656599366,
  0.13393960528010046,
  0.027752752562134365,
  0.06822357388189883,
  0.09415349021629964,
  0.021604855129709645],
 [0.11642779312049328,
  0.03500871204275638,
  0.2001644266776521,
  0.22853582645026913,
  0.05705961365388699,
  0.01448423244314698],
 [0.21770121545548302,
  0.47430974073754356,
  0.2583442218816916,
  0.19753170323817715,
  0.27416070244243596,
  0.1788910881387213],
 [0.20520339537895843,
  0.34453

In [8]:
from scipy.stats import spearmanr

comp_arr = np.array(comp_arr)
rank_algs = ["faithfulness (itself)", "rank_based_fairthfulness:sum", "rank_based_fairthfulness:percentile", "rank_based_fairthfulness:avg", "rank_based_fairthfulness:inverse"]
# rank_algs = ["NRC"]

pearson_corr = [np.corrcoef(comp_arr[:, 0], comp_arr[:, i])[0, 1] for i in range(1, comp_arr.shape[1])]
spearman_corr = [spearmanr(comp_arr[:, 0], comp_arr[:, i]).correlation for i in range(1, comp_arr.shape[1])]
kendall_corr = [kendalltau(comp_arr[:, 0], comp_arr[:, i]).correlation for i in range(1, comp_arr.shape[1])]

correlation_df = pd.DataFrame({
    'Rank Algorithm': rank_algs,
    'Pearson Correlation': pearson_corr,
    'Spearman Correlation': spearman_corr,
    'Kendall Correlation': kendall_corr
})

print(dataset_name)
print("The following are the correlations between the faithfullness metric and... (abs fixed)")
display(correlation_df)

PAKDD2010
The following are the correlations between the faithfullness metric and... (abs fixed)


,Rank Algorithm,Pearson Correlation,Spearman Correlation,Kendall Correlation
0,faithfulness (itself),0.911762,0.895730,0.713534
1,rank_based_fairthfulness:sum,0.617967,0.542093,0.377659
2,rank_based_fairthfulness:percentile,0.631631,0.577506,0.405708
3,rank_based_fairthfulness:avg,0.628643,0.572021,0.399599
4,rank_based_fairthfulness:inverse,0.591073,0.531733,0.369543
